#### Repo : [Click Here](https://github.com/shadmehr-salehi/AI-Hackathon-2023)

### Used Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.metrics import accuracy_score , f1_score

In [4]:
df_train = pd.read_csv('D:/Final Heckathon/2/train_data.csv')

In [5]:
df_train.isna().sum()

title          0
description    0
tags           2
dtype: int64

In [7]:
df_train.dropna(inplace=True)

In [8]:
df_train['tags'] = df_train['tags'].str.replace('.*اقتصاد.*', 'اقتصادی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*اجتماع.*', 'اجتماعی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*استان.*', 'ایران_استانها', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*ایران.*', 'ایران_استانها', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*ايران.*', 'ایران_استانها', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*ملل.*', 'بین الملل', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*جهان.*', 'بین الملل', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*آلمان.*', 'بین الملل', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*سیاس.*', 'سیاسی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*ورزش.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*فرهنگ.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*علم.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*دانش و فناوری.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*رسانه ها.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*ایسنا.*', 'اجتماعی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*دانش و محیط زیست.*', 'علمی_فرهنگی_ورزشی', regex=True)
df_train['tags'] = df_train['tags'].str.replace('.*بازار.*', 'اقتصادی', regex=True)

In [9]:
names_to_drop = ["علمی_فرهنگی_ورزشی", "سیاسی", "اقتصادی", "اجتماعی", "ایران_استانها", "بین الملل"]
df_train = df_train[df_train['tags'].isin(names_to_drop)]


tags_num = []
for row in df_train['tags']:
    if (row == 'اقتصادی'):
        tags_num.append(1)
    elif ( row  == 'اجتماعی'):
        tags_num.append(0)
    elif (row == 'ایران_استانها'):
        tags_num.append(2)
    elif (row == 'بین الملل'):
        tags_num.append(3)
    elif (row == 'سیاسی'):
        tags_num.append(4)
    elif (row == 'علمی_فرهنگی_ورزشی'):
        tags_num.append(5)

df_train['tags_num'] = tags_num

df_train.drop(['tags'], axis=1, inplace=True)

In [11]:
df_train['text'] = df_train['title'] + ' ' + df_train['description']

In [13]:
train_df, test_df = train_test_split(df_train, test_size=0.2, random_state=42)

In [15]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['tags_num'])
test_labels = label_encoder.transform(test_df['tags_num'])

In [29]:
max_words = 10000 
max_sequence_length = 100 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['text'])
X_train_sequences = tokenizer.texts_to_sequences(train_df['text'])
X_test_sequences = tokenizer.texts_to_sequences(test_df['text'])
X_train = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test = pad_sequences(X_test_sequences, maxlen=max_sequence_length)


In [39]:
embedding_dim = 3500
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_sequence_length),
    Conv1D(256, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [40]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
model.fit(X_train, train_labels, epochs=10, validation_split=0.1)

Epoch 1/10
235/235 [==============================] - 16s 61ms/step - loss: 0.6730 - accuracy: 0.7625 - val_loss: 0.4663 - val_accuracy: 0.8216
Epoch 2/10
235/235 [==============================] - 14s 60ms/step - loss: 0.1648 - accuracy: 0.9487 - val_loss: 0.4889 - val_accuracy: 0.8539
Epoch 3/10
235/235 [==============================] - 14s 61ms/step - loss: 0.0264 - accuracy: 0.9927 - val_loss: 0.6242 - val_accuracy: 0.8515
Epoch 4/10
235/235 [==============================] - 14s 61ms/step - loss: 0.0261 - accuracy: 0.9916 - val_loss: 0.5662 - val_accuracy: 0.8659
Epoch 5/10
235/235 [==============================] - 14s 60ms/step - loss: 0.0173 - accuracy: 0.9952 - val_loss: 0.9054 - val_accuracy: 0.8251
Epoch 6/10
235/235 [==============================] - 14s 60ms/step - loss: 0.0697 - accuracy: 0.9813 - val_loss: 0.7714 - val_accuracy: 0.8335
Epoch 7/10
235/235 [==============================] - 14s 61ms/step - loss: 0.0138 - accuracy: 0.9963 - val_loss: 0.9114 - val_accuracy:

In [43]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = f1_score(test_labels, predicted_labels , average='weighted')
print("F1 Score:", accuracy)

F1 Score: 0.8448822540465144
